### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Fall 2018 ###

# Drawing Fair Voting Districts #

#### Apivich Hemachandra (email: ahemachandra@wisc.edu, ID: 908 032 6391)

### Table of Contents ###

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-Model)
   1. [Population Model](#2.1-Population-Model)
   1. [Drawing Voting Districts](#2.2-Drawing-Voting-Districts)
       1. [Block Assignment Scheme without Graphs](#2.2.1.-Block-Assignment-Scheme-without-Graphs)
       1. [Block Assignment Scheme with Graphs](#2.2.2.-Block-Assignment-Scheme-with-Graphs)
       1. [Line Drawing Scheme](#2.2.3.-Line-Drawing-Scheme)
   1. [Gerrymandering Objectives](#2.3.-Gerrymandering-Objectives)
1. [Solution](#3.-Solution)
   1. [Global Variables](#3.1-Global-Variables)
   1. [Data Generation](#3.2-Data-Generation)
       1. [Clustered Voters Distribution](#3.2.1-Clustered-Voters-Distribution)
       1. [Random Voters Distribution](#3.2.2-Random-Voters-Distribution)
   1. [District Drawing](#3.3.-District-Drawing)
       1. [Block Assignment Scheme without Graphs](#3.3.1.-Block-Assignment-Scheme-without-Graphs)
       1. [Block Assignment Scheme with Graphs](#3.3.2.-Block-Assignment-Scheme-with-Graphs)
       1. [Line Drawing Scheme](#3.3.3.-Line-Drawing-Scheme)
1. [Results and Discussion](#4.-Results-and-Discussion)
    1. [Geometry of Solutions](#4.1.-Geometry-of-Solutions)
    1. [Quantitative Comparisons of Different Methods](#4.2.-Quantitative-Comparisons-of-Different-Methods)
        1. [Comparison of the Block Models](#4.2.1.-Comparison-of-the-Block-Models)
        1. [Efficiency Gap](#4.2.2.-Efficiency-Gap)
        1. [Variation of $bias$ on solution of graph method without graphs](#4.2.3.-Variation-of-$bias$-on-solution-of-graph-method-without-graphs)
    1. [Discussion on Different Schemes](#4.3.-Discussion-on-Different-Schemes)
    1. [Gerrymandering](#4.4.-Gerrymandering)
1. [Conclusion](#5.-Conclusion)
1. [Sources](#6.-Sources)

***

## 1. Introduction ##

One of the issues about election processes is the problem of drawing fair voting districts. Fair voting districts could be defined as one where the representatives that are chosen by the election matches as closely to the actual voter behaviours of the actual populations as possible. An example can be seen in the diagram below labelled (b), where there are just over one-third green voters, and the districts are divided such that one of the three districts are won by green voters.

The opposite action of drawing fair voting districts is gerrymandering. Gerrymandering is an act of manipulating the boundaries of election districts such that it favours one of the parties. There are two main ways that political parties can gerrymander to favour their own parties:

- _Cracking_ - the process of assigning similar voters to different districts so that they form a minority in the different districts and cannot win the election. This is a useful technique for parties with *majority* voters. This is demonstrated in the diagram below labelled (a), where the green voters are distributed amongst the different districts and becomes a minority group in all of the districts.
- _Packing_ - the process of grouping similar voters together so that there could be some districts which contain only like-minded voters, and leave other districts more competitive with less of such voters. This is a useful technique for parties with *minority* voters. This is demonstrated by the diagram below labelled (c), where the blue voters are all grouped in one district, leaving the rest of the blue voters a minority group in the other districts.

![Pic](https://upload.wikimedia.org/wikipedia/commons/d/d7/Gerrymandering_9-6.png)

(_Image above taken from_ https://upload.wikimedia.org/wikipedia/commons/d/d7/Gerrymandering_9-6.png)

Often, these districts will be very carefully crafted in order to achieve the cracking or packing desired. They can change the outcome of elections without actually having to change the behaviours of voters themselves. There exists examples of electoral districts in the United States where gerrymandering is present. An example include a district in North Carolina which groups voters of similar race together in the same district 

![NC District](https://upload.wikimedia.org/wikipedia/commons/e/e8/North_Carolina_12th_Congressional_District_%28National_Atlas%29.gif) 

(_Image above taken from_ https://upload.wikimedia.org/wikipedia/commons/e/e8/North_Carolina_12th_Congressional_District_%28National_Atlas%29.gif)

There are a couple of ways of countering gerrymandering, such as to use computer algorithms which would draw voting districts of similar population sizes while disregarding the demographics of the people entirely (which may create unfair voting districts but would be by pure chances), or to set up independent committees who may draw voting districts such that it considers the demographics and make it more fair (which is what we will do).

To quantify how fair a voting district is, we can use the concept of *wasted votes*, which are votes that is either
- For the minority party, or
- For the majority party above what they need in order to get the majority in the district (i.e. if a party gets 70 out of 100 possible votes in an area, they have 19 wasted votes because they only need 51 to get a majority there, so 19 of the votes is just extra).
Quantitiatively, a fair voting district is one with the lowest *efficiency gap*, which (assuming we are in a two-party system) is the difference between the wasted votes of the two parties divided the number of total votes. If there is gerrymandering for a certain party, the efficiency gap will be higher.

In this project, we will devise different methods of drawing voting districts given a smaller, self-generated sample population model in order to minimise the efficiency gap, or to make the voting districts as fair as possible. Population model will be generated both purely randomly, and with a model which creates clusters of similar voters. We will experiment with drawing district shapes by dividing town into "blocks" and assigning each of these blocks to a different district, and also by drawing straight lines and divide a town into slices. We will also experiment with an alternative objective of the problem where we aim to gerrymander the districts such that it is in favour of one of the two parties in the election, by trying to maximise the number of districts won by one of the parties.

After devising a model, we will discuss the general geometry of the districts that is drawn and also compare how the model performs against each other in terms of running time and minimal efficiency gap that we are able to get.

***

## 2. Mathematical Model ##

### 2.1 Population Model ###

In reality, exact voting behaviours of people living in certain locations cannot be determined, but they are rather guessed according to the demographics of the population (i.e. by gender, race, age, education, etc.). However, for the purpose of our problem, we will generate our own sets of data which will represent the exact voting behaviours of people at given locations.

We will define some variables regarding our model of the city as follows.
- Let the city be a square shaped of size $n$ by $n$.
- Assume the city is bipartisan, i.e. only consisting of two parties. We will have two types of voters, one who will intend to vote for Party A and one which will vote for Party B. Let $Anum$ be the number of voters for Party A, and $Bnum$ be the number of voters for Party B. We also define $vts$ to be $Anum + Bnum$ for convienience. For our model, it doesn't matter which party has majority voters or minority voters.
- Assume we want to divide the city into some number of districts defined by the parameter $dist$.

We can generate the voter distribution by generating the location a certain voter will be at. There are two schemes we can use to generate these voters.

#### Clustered Voters Distribution ####

In this model, we select two points to be the "focus" point for the voters of Party A and for Party B. We then use the `randn()` command on Julia and generate a random number based on the normal distribution, where the mean is at the "focus" point and the standard deviation is determined by the user. 

Since `randn()` only generates a number from the normal distribution centred at zero and with a standard deviation of 1, to generate a random point for our voters we need to manipulate the randomly generated number. The mathematical expression for the randomly generated point for the voter in Party A is given by 
$$ \Big( sdv_A \cdot \texttt{randn()} + x_{A},  \quad sdv_A \cdot \texttt{randn()} + y_{A} \Big)$$
while the generated point for the minority voter is given by 
$$ \Big( sdv_B \cdot \texttt{randn()} + x_{B},  \quad sdv_B \cdot \texttt{randn()} + y_{B} \Big)$$
where $sdv_A$ and $sdv_B$ are the standard deviations for distribution for the two types of voters, and $(x_{A}, y_{A})$ and $(x_{B}, y_{B})$ are the focus points for the majority and the minority voters respectively.

#### Random Voters Distribution ####

In this scheme, the coordinates of the voters are generated randomly using the `rand()` command on Julia. The coordinate of a voter would be generated by the expression 
$$ \Big( n \cdot \texttt{rand()},  \quad  n \cdot \texttt{rand()} \Big)$$
where the factor of $n$ is multiplied since `rand()` only generates numbers between 0 and 1. The probability that the voters is at any location in the town is equal.

In larger towns, this model is more unrealistic as similar voters will tend to cluster together in real life. However this model will still provide an interesting case for our problem when analysing the limits of the models and how well it can draw district lines when there are no clear patterns in the voters distribution.

We will store the coordinates of the voters in arrays called $Alist$ and $Blist$, where the coordinate of the $i$th voter of the A party is $(Alist[i,1], Alist[i,2])$ and the coordinate of the $j$th voter of the B party is $(Blist[j,1], Blist[j,2])$.

### 2.2 Drawing Voting Districts ###

When we assign the voters to a district, there are a couple of constraints that we have to take into account. These are:

1. The voters cannot be assigned to more than one district.
2. The number of voters in a district must be roughly equal. Since it is not practical to make the size of all districts be equal to each other, we can use some parameter $pd$ which will be the maximum percentage difference between actual district size and the ideal district size (which is equal to total voters divided by the number of districts).
3. A certain district must be connected and isn't split into two parts (i.e. you can walk between two points in the district without leaving the district - the city doesn't have to be convex though). 

We will consider the voting districts being drawn in three ways (as you will also see my level of creativity in naming the schemes isn't that high). We will have to ensure that the conditions above are satisfied. Many of the constraints in the models will be repeated because the same conditions above must be satisfied.

All of the models below are mixed integer programs (MIP).

#### 2.2.1. Block Assignment Scheme without Graphs ####

In the block assignment scheme, we divide the town into $blk \times blk$ blocks. We then assign each of the blocks to a certain district, keeping in mind the conditions above.

Suppose we have a city of some size. We will divide them into blocks and label their coordinates so it looks something like the image below.

<img src="img/blk1.png" width="400">

To record how many of each voters are in each of the blocks, we define a new set of variables. We will define two new arrays $Ablock$ and $Bblock$, where $Ablock[i,j]$ and $Bblock[i,j]$ are the number of voters of Party A and Party B in block $(i,j)$. These two arrays can be formed by looping through each voters in $Alist$ and $Blist$ and figuring out which block the voters will have to be in.

For the block assignment, we define a variable $X$ where 
$$X[i,j,k] = \begin{cases}
1 \quad \text{if block $(i,j)$ is in district $k$,}\\
0 \quad \text{if otherwise.} \end{cases}$$
Because a block can only be allocated to one district this means that we have the constraint $\sum_{k=1}^{dist} X[i,j,k] = 1$ for all blocks $(i,j)$.

We also define some extra variables which will constrain our districts further. These are:
- $vA[i]$ which are the number of votes for Party A in district $i$,
- $vB[i]$ which are the number of votes for Party B in district $i$,
- $voters[i]$ which is just the total voters in district $i$, or equal to $vA[i] + vB[i]$,
- $xedge[i,j,k]$ which is 1 if blocks $(i,j)$ and $(i+1,j)$ are allocated to district $k$ and is 0 otherwise,
- $yedge[i,j,k]$ which is 1 if blocks $(i,j)$ and $(i,j+1)$ are allocated to district $k$ and is 0 otherwise,
- $Awon[i]$ which is 1 if Party A wins election in district $i$ and is 0 otherwise,
- $Bwon[i]$ which is 1 if Party B wins election in district $i$ and is 0 otherwise,
- $wA[i]$ which is the number of wasted votes by Party A in district $i$,
- $wB[i]$ which is the number of wasted votes by Party B in district $i$,
- $gap$ which is the absolute difference between wasted votes of the two parties.

Why we need these variables will become apparent when we discuss the rest of the formulation of the problem.

The first thing we have to do is to work out the number of voters belonging to each of the district. This can be done simply using the variable $X$. To find the number of Party A voters in some district $k$, we can sum the expression $X[i,j,k] \cdot Ablock[i,j,k]$ over all $i$ and $j$. This will be $vA[i]$. An analogous summation of $X[i,j,k] \cdot Bblock[i,j,k]$ over all $i$ and $j$ would give us the total Party B voters in district $k$. This will be $vB[i]$. From this, we can also say the total voters in a district is given by $voters[i] = vA[i] + vb[i]$.

The size of each district must be about the same. We defined the variable $pd$ representing the maximum deviation allowed in district size. If all districts were equal then there will be $\dfrac{vts}{dist}$ voters in each district. With the error factors, one district can have between $\Big(\dfrac{vts}{dist}\Big)(1-pd)$ and $\Big(\dfrac{vts}{dist}\Big)(1+pd)$ voters. This results in the constraint $\Big(\dfrac{vts}{dist}\Big)(1-pd) \leq voters[i] \leq \Big(\dfrac{vts}{dist}\Big)(1+pd)$ for all districts $i$.

Once we find the total number of voters in each party, we can find out which party won each district. This can be done by comparing values in $vA$ and $vB$ with each other. We say that for a district $i$, Party A won ($Awon[i] = 1$, $Bwon[i] = 0$) iff $vA[i] > vB[i]$ or iff $vA[i] \geq vB[i] + 1$, and Party B won ($Awon[i] = 0$, $Bwon[i] = 1$) iff $vA[i] < vB[i]$ or iff $vA[i] + 1 \leq vB[i]$. Note that we will ensure a strictly greater/less than constraint so that we do not have any ties in the election and so each district will have one definite winner. 

To enforce the two conditions above, we can say that $vA[i] - vB[i] - 1 \geq -mar\cdot Bwon[i]$ and $-vA[i] + vB[i] - 1 \geq -mar\cdot Awon[i]$ where $mar = \Big(\dfrac{vts}{dist}\Big)(1+pd)$ which is the maxmimum winning margin possible in a district given possible size of a district. The two constraints above are logic constraints. We can also impose a further constraint that $Awon[i] + Bwon[i] = 1$ to ensure that there must be exactly one winner in a given district.

To calculate the number of wasted votes from each party, we remind ourselves that wasted votes are either votes for the losing party or votes on top of what was needed to win. So as an example, consider the wasted votes for Party A in district $i$. 

- If they lost in district $i$, then the wasted votes will be the total votes in district $i$ for Party A. This value will be $vA[i]$. This term will only matter when Party B wins or $Bwon[i]=1$.
- If they won in district $i$, then the wasted votes will be the votes above the majority needed. The number of votes needed for majority can be said to be half of the total votes in the district, or $\dfrac{voters[i]}{2}$ (we will not use the "half-plus-one" majority method because we'd need to distinguish between odd or even number of voters in the district which complicates the problem further without increased precision in solution). So the wasted votes in this case will be $vA[i] - \dfrac{voters[i]}{2}$. This term will only matter when Prty A wins or $Awon[i]=1$.

So we can say the total wasted votes for Party A in district $i$ is given by $vA[i]\cdot Bwon[i] + Awon[i]\Big( vA[i] - \dfrac{voters[i]}{2}\Big)$. We can see that only one of the two term in the expression will contribute to wasted votes depending on who won in the district. For the total number of wasted votes $wA$ across all districts we sum this term over all $i$. An analogous method can be used to find the wasted votes for Party B.

We finally can define the efficiency gap of our model. As a reminder, the efficiency gap is the difference between wasted votes of the two parties, divided by the total number of votes. Mathematically, this means that $gap = \dfrac{\left| wA-wB \right|}{vts}$. To code this in Julia, this means that $gap \geq \dfrac{wA-wB}{vts}$ and $gap \geq \dfrac{wB-wA}{vts}$. This value must be minimised in our problem.

However, we still have to ensure that our district is connected. For this scheme, we will do so empirically by making connectivity of districts be another objective. 

We will use the variables $xedge$ and $yedge$. We remind ourselves that $xedge[i,j,k]$ is only 1 if district $(i,j)$ and $(i+1,j)$ are both in district $k$. As a constraint this means that $2\cdot xedge[i,j,k] \leq X[i,j,k] + X[i+1,j,k]$. We also have $yedge[i,j,k]$ is only 1 if district $(i,j)$ and $(i,j+1)$ are both in district $k$. As a constraint this means that $2\cdot yedge[i,j,k] \leq X[i,j,k] + X[i,j+1,k]$. 

If the districts are more separated then not many terms in $xedge$ and $yedge$ will be 1 because not many neighbouring blocks are of the same district. If the districts are mostly connected, more edges will satisfy conditions above, and more terms in $xedge$ and $yedge$ will be 1. So if we want districts that are mostly connected, we will want $conn = \sum_{i,j,k} xedge[i,j,k] + \sum_{i,j,k} yedge[i,j,k]$ to be as high as possible. This can be treated as another objective we need. 

So our overall objective consists of two parts - part which minimises efficiency gap and the part which maximises connectivity of districts. So our objective would be to minimise $gap - bias\cdot conn$, where $bias \geq 0$ is the bias factor on how much we care about connected districts.

The final model of our optimisation is therefore

\begin{align}
\text{with variables} \quad & X[i,j,k] && \forall i,j \in 1:blk, \forall k \in 1:dist \\ 
& vA[k] = \sum_{i=1}^{blk} \sum_{j=1}^{blk} X[i,j,k] \cdot Ablock[i,j] && \forall k \in 1:dist \\ 
& vB[k] = \sum_{i=1}^{blk} \sum_{j=1}^{blk} X[i,j,k] \cdot Bblock[i,j] && \forall k \in 1:dist \\ 
& voters[k]=vA[k]+vB[k] && \forall k \in 1:dist \\
& xedge[i,j,k] && \forall i=1:(blk-1), \forall j=1:blk, \forall k=1:dist \\
& yedge[i,j,k] && \forall i=1:blk, \forall j=1:(blk-1), \forall k=1:dist \\
& Awon[k], Bwon[k]&& \forall k=1:dist \\
& wA = \sum_{k=1}^{dist} Bwon[k] \cdot vA[k] + Awon[k]\Big( vA[k] - \dfrac{voters[k]}{2}\Big) \\
& wB = \sum_{k=1}^{dist} Awon[k] \cdot vB[k] + Bwon[k]\Big( vB[k] - \dfrac{voters[k]}{2}\Big) \\
& gap \geq 0 \\
& conn = \sum_{i=1}^{blk-1} \sum_{j=1}^{blk} \sum_{k=1}^{dist} xedge[i,j,k] + \sum_{i=1}^{blk} \sum_{j=1}^{blk-1} \sum_{k=1}^{dist} yedge[i,j,k] \\
\text{minimise} \quad & gap - bias\cdot conn \\
\text{subject to} \quad & \sum_{k=1}^{dist} X[i,j,k] = 1 && \forall i,j \in 1:blk \\
& vA[k] - vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Bwon[k] && \forall k=1:dist \\
& -vA[k] + vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Awon[k] && \forall k=1:dist \\
& Awon[k] + Bwon[k] = 1 && \forall k=1:dist \\
& 2\cdot xedge[i,j,k] \leq X[i,j,k] + X[i+1,j,k] && \forall i=1:(blk-1), \forall j=1:blk, \forall k=1:dist \\
& 2\cdot yedge[i,j,k] \leq X[i,j,k] + X[i,j+1,k] && \forall i=1:blk, \forall j=1:(blk-1), \forall k=1:dist \\
& \Big(\dfrac{vts}{dist}\Big)(1-pd) \leq voters[k] \leq \Big(\dfrac{vts}{dist}\Big)(1+pd) && \forall k=1:dist \\
& gap \geq \dfrac{wA-wB}{vts} \\
& gap \geq \dfrac{wB-wA}{vts}
\end{align}

(Note: I borrowed some Julia notation by using $1:n$ to represent all integers between $1$ and $n$ inclusive.)

#### 2.2.2. Block Assignment Scheme with Graphs ####

In the model above, we get the solution where the districts are all connected by defining the objective such that the objective function is lower if the districts are more connected with each other. We can instead use properties of graphs to make the connectivity of the districts a constraint rather than a preference in the objective, at a cost of more calculation time (see results).

Consider one of the districts and all of the blocks that is allocated to this district. We can represent the blocks in the districts using nodes of a graph, and we can let there be an edge connecting two of the nodes if the blocks corresponding to those nodes border each other. The district that we have then would form a graph. For simplicity, let the nodes representing each of the blocks in a certain district belong in a set $V$ and the valid edges belong to a set $E$.

<img src="img/bwg1.png" width="800">

_Left: A sample of a district and a possible district division. Right: An example of a district and the possible graph it forms, with all nodes for such district (represented with blue points) and all possible edges connecting adjacent districts (represented in red)._

To check if all of the blocks are connected to each other, we see whether we can form a tree connecting all the nodes in $V$ using some of the edges in $E$. The tree formed must be all connected (i.e. is not a forest) and must not form a cycle. 

<img src="img/bwg2.png" width="400">

_Above: an example of an invalid district division. Because there is no way to form a single tree connecting all of the districts, the blocks chosen for the district are not all connected. However if the blocks (1,1) and (1,2) are taken out, then we can form a tree, hence get a valid district._

Both of these conditions can be formulated in our linear program. First, we use the property that a tree of size $n$ must have $n-1$ edges. This means that out of the $|E|$ edges we have, we must select exactly $|V|-1$ of the edges in our tree.

The property above may still create solutions which is a forest. However, with the constraint on the number of edges in our subset of edges, we know that if the graph is a forest, there must be a cycle somewhere (the reason is if we have a tree and we move one of the edge to form a cycle, the graph would be disconnected where the old edge used to be). So we will also have to add a constraint for cycle prevention which at the same time will make sure there is only one tree.

We can prevent cycle by the following. Let an edge act as a source which will have a total flow outward of 2. This flow will be distributed to the two nodes connected to this edge (not necessarily in integer amounts). 

<img src="img/bwg3.png" width="200">

_Above: illustration of flow from an edge, divided to the two nodes it is connected to._

For there to be no cycle, we limit that the total flow into any of the nodes in $V$ must be strictly less than 2. This can be seen to work by considering a graph with a cycle, and trying to divide the flow from all of the edges. There is no way to keep all the net flow into every node less than 2.

We will write this as a linear program. We will define some variables as below.
- $xedge[i,j,k]=1$ if there is an edge from block $(i,j)$ to block $(i+1,j)$ in tree for district $k$ and $0$ otherwise,
- $yedge[i,j,k]=1$ if there is an edge from block $(i,j)$ to block $(i,j+1)$ in tree for district $k$ and $0$ otherwise,
- $flow[i,j,k,m]$ is the flow into block $(i,j)$ in graph of district $k$ from some direction $m$. So if for some block $(i,j)$ we have four bordering blocks in the same district (i.e. connected to 4 edges), then there could be flow into the node from the edges, which corresponds to the values in $flow$.

If two neighbouring blocks are both not in some district $k$, then there can be no edge between them. This is a logic constraint $2\cdot xedge[i,j,k] \leq X[i,j,k] + X[i+1,j,k]$. Analogous constraint can be formed for $yedge$.

Then we say if an edge exists then it will produce a flow of 2 into the two nodes connected to it. We can write this as $2 \cdot xedge[i,j,k] = flow[i,j,k,1] + flow[i+1,j,k,2]$ and $2\cdot yedge[i,j,k] = flow[i,j,k,3] + flow[i,j+1,k,4]$. We can see that the variable $m$ is just to specify the flow into a node from a certain direction.

We can then say that for a certain node, the total flow must be less than 2, or that $\sum_{m} flow[i,j,k,m] < 2$, or $\sum_{m} flow[i,j,k,m] \leq 2 - \epsilon$ where $\epsilon$ is a small number.

Finally, for a valid tree representing district $k$, the number of edges must be one less than the number of nodes. This means that $\sum_{i,j} xedge[i,j,k] + \sum_{i,j} yedge[i,j,k] + 1 = \sum_{i,j} X[i,j,k]$ for some District $k$.

When these changes are made to the model from the original block scheme (Section 2.2.1), our modified model is now therefore

\begin{align}
\text{with variables} \quad & X[i,j,k] && \forall i,j \in 1:blk, \forall k \in 1:dist \\ 
& vA[k] = \sum_{i=1}^{blk} \sum_{j=1}^{blk} X[i,j,k] \cdot Ablock[i,j] && \forall k \in 1:dist \\ 
& vB[k] = \sum_{i=1}^{blk} \sum_{j=1}^{blk} X[i,j,k] \cdot Bblock[i,j] && \forall k \in 1:dist \\ 
& voters[k]=vA[k]+vB[k] && \forall k \in 1:dist \\
& xedge[i,j,k] && \forall i=1:(blk-1), \forall j=1:blk, \forall k=1:dist \\
& yedge[i,j,k] && \forall i=1:blk, \forall j=1:(blk-1), \forall k=1:dist \\
& flow[i,j,k,m] && \forall i,j \in 1:blk, \forall k \in 1:dist, \forall m \in 1:4 \\
& Awon[k], Bwon[k]&& \forall k=1:dist \\
& wA = \sum_{k=1}^{dist} Bwon[k] \cdot vA[k] + Awon[k]\Big( vA[k] - \dfrac{voters[k]}{2}\Big) \\
& wB = \sum_{k=1}^{dist} Awon[k] \cdot vB[k] + Bwon[k]\Big( vB[k] - \dfrac{voters[k]}{2}\Big) \\
& gap \geq 0 \\
\text{minimise} \quad & gap\\
\text{subject to} \quad & \sum_{k=1}^{dist} X[i,j,k] = 1 && \forall i,j \in 1:blk \\
& vA[k] - vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Bwon[k] && \forall k=1:dist \\
& -vA[k] + vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Awon[k] && \forall k=1:dist \\
& Awon[k] + Bwon[k] = 1 && \forall k=1:dist \\
& 2\cdot xedge[i,j,k] \leq X[i,j,k] + X[i+1,j,k] && \forall i=1:(blk-1), \forall j=1:blk, \forall k=1:dist \\
& 2\cdot yedge[i,j,k] \leq X[i,j,k] + X[i,j+1,k] && \forall i=1:blk, \forall j=1:(blk-1), \forall k=1:dist \\
& 2\cdot xedge[i,j,k] = flow[i,j,k,1] + flow[i+1,j,k,2] && \forall i=1:(blk-1), \forall j=1:blk, \forall k=1:dist \\
& 2\cdot yedge[i,j,k] = flow[i,j,k,3] + flow[i,j+1,k,4] && \forall i=1:blk, \forall j=1:(blk-1), \forall k=1:dist \\
& \sum_{m=1}^{4} flow[i,j,k,m] \leq 2-\epsilon && \forall i,j = 1:blk, \forall k=1:dist \\
& \sum_{i=1}^{blk-1} \sum_{j=1}^{blk}  xedge[i,j,k] + \sum_{i=1}^{blk} \sum_{j=1}^{blk-1} yedge[i,j,k] + 1 = \sum_{i=1}^{blk} \sum_{j=1}^{blk} X[i,j,k] && \forall k=1:dist \\
& \Big(\dfrac{vts}{dist}\Big)(1-pd) \leq voters[k] \leq \Big(\dfrac{vts}{dist}\Big)(1+pd) && \forall k=1:dist \\
& gap \geq \dfrac{wA-wB}{vts} \\
& gap \geq \dfrac{wB-wA}{vts}
\end{align}

Constraints 5 to 10 above are the consraints we added to ensure the graphical representation of our solution matches constraints we want. The rest of the constraints are the same as the model from Section 2.2.1.

#### 2.2.3. Line Drawing Scheme ####

In this scheme, to divide the town up into $dist$ districts, we will draw $dist-1$ strainght lines across the town which will cut it up into districts. 

Much of the formulation will use variables similar to what has already been used above. There will be some differences though because we will have to track the districts of individual voters rather than the districts of a group of voters in a block. We will define a variable $XA[i,k]$ where
$$XA[i,k] = \begin{cases}
1 \quad \text{if the $i$th voter of Party A is in district $k$,}\\
0 \quad \text{if otherwise,} \end{cases}$$
and $XB[i,k]$ where
$$XB[j,k] = \begin{cases}
1 \quad \text{if the $j$th voter of Party B is in district $k$,}\\
0 \quad \text{if otherwise.} \end{cases}$$
This gives rise to the constraint $\sum_k XA[i,k] = 1$ for all possible $i$ and  $\sum_k XB[j,k] = 1$ for all possible $j$ because each voters can only be in one district.

A straight line drawn in the town can be characterised using a y-intercept and a slope. We will define variables $slp$ and $yint$ such that $m$th line will have the slope of $slp[m]$ and the y-intercept of $yint[m]$, and have the equation $y=slp[m] \cdot x + yint[m]$. Bounds on the y-intercept and on the slope will not be placed (other constraints on district size will ensure we draw reasonable lines). 

To keep our division of area simple, we will apply a constraint that the line does not cross with each other in our city (when $0 \leq x \leq n$). Let's say we want line $m$ to be below line $m+1$ at all times in the city and not cross each other. Then that means at $x=0$ and at $x=n$, the point on line $m$ must be below line $m+1$, or $m$ must have a lower $y$ coordinate than line $m+1$ at the two points of $x$. We can enforce this constraint by saying that $yint[m] \leq yint[m+1]$ and $n\cdot slp[m] + yint[m] \leq n\cdot slp[m+1] + yint[m+1]$ for all $m = 1,...,dist-1$.

Now we have to figure out which voters belong to which district given our lines drawn. If a voter is in district $m$, it must mean they are above line $m-1$ and below line $m$ (where we say line $0$ is the x-axis and line $m$ is the line $y=n$). If the voter is below line $m$ then they cannot be in district $m+1$. Similarly, if the voter is above line $m$, they cannot be in district $m$. This can be expressed as a logic constraint.

<img src="img/line1.png" width="400">

_Above: a diagram for a sample voter. If the voter is in district 2, they will be between lines 2 and 3. If the voter is in district 2, they must be above line 1, and must be below lines 2 and 3. These statements need to be added as a constraint._

Let's consider the $i$th voter of Party A. Their coordinate in the town is given by $(Alist[i,1], Alist[i,2])$. 

First we can see if the voter is below line $m$. If they are, then they cannot be in district $m+1$. Symbolically, this means that if $slp[m] \cdot Alist[i,1] + yint[m] \geq Alist[i,2]$, then $XA[i,m+1] = 0$. As a logic constraint, this is $(slp[m]\cdot Alist[i,1] + yint[m]) \leq Alist[i,2] + n(1-XA[i,m+1]) - \epsilon \cdot XA[i,m+1]$. While the left hand side doesn't have an actual upper bound (the line can go as low as one want), we can assume that for a reasonable line, the upper bound on the left hand side is $n$. We also let $\epsilon$ be some small number.

We also need to check if the voter lies above line $m$. If they are then they cannot be in district $m$. Symbolically, this means that if $slp[m] \cdot Alist[i,1] + yint[m] \leq Alist[i,2]$ then $XA[i,m] = 0$. As a logic constraint, this says that $slp[m] \cdot Alist[i,1] + yint[m] - Alist[i,2] \geq \epsilon \cdot XA[i,m] - n(1-XA[i,m])$. Again, the left hand side does not have an actual lower bound, so we can impose a lower bound of $-n$ on it. 

Similar constraints also needed to be applied for the voters of Party B.

Note that above, we had to set our own upper and lower bounds on $slp[m] \cdot Alist[i,1] + yint[m] - Alist[i,2]$ which would actually have no bounds in real life (e.g. if our y-intercept is beyond $n$, which is not unreasonable as long as it cuts the city somewhere). We could have avoid upper/lower bounds in our constraint, but it would require nonlinear constraints which would have slowed down calculation time by quite a bit (I found this out when playing around with the code) and not give us any significantly better results.

We will define more variables for our problem similar to above, which are the following.

- $vA[i]$ which are the number of votes for Party A in district $i$,
- $vB[i]$ which are the number of votes for Party B in district $i$,
- $voters[i]$ which is just the total voters in district $i$, or equal to $vA[i] + vB[i]$,
- $Awon[i]$ which is 1 if Party A wins election in district $i$ and is 0 otherwise,
- $Bwon[i]$ which is 1 if Party B wins election in district $i$ and is 0 otherwise,
- $wA[i]$ which is the number of wasted votes by Party A in district $i$,
- $wB[i]$ which is the number of wasted votes by Party B in district $i$,
- $gap$ which is the absolute difference between wasted votes of the two parties.

These variables are formulated in exactly the same way as in the previous schemes, with the only change being that now $vA[k] = \sum_i XA[i,k]$ and $vB[k] = \sum_i XB[i,k]$.

The objective of our problem is to minimise the efficiency gap, or to minimise $gap$.

The final model of our optimisation is therefore

\begin{align}
\text{with variables} \quad & XA[i,k] && \forall i \in 1:Anum, \forall k \in 1:dist \\ 
& XB[i,k] && \forall i \in 1:Bnum, \forall k \in 1:dist \\ 
& slp[k], yint[k] && \forall k \in 1:(dist-1) \\
& vA[k] = \sum_{i=1}^{Anum} XA[i,k] && \forall k \in 1:dist \\ 
& vB[k] = \sum_{i=1}^{Bnum} XB[i,k] && \forall k \in 1:dist \\ 
& voters[k]=vA[k]+vB[k] && \forall k \in 1:dist \\
& Awon[k], Bwon[k]&& \forall k=1:dist \\
& wA = \sum_{k=1}^{dist} Bwon[k] \cdot vA[k] + Awon[k]\Big( vA[k] - \dfrac{voters[k]}{2}\Big) \\
& wB = \sum_{k=1}^{dist} Awon[k] \cdot vB[k] + Bwon[k]\Big( vB[k] - \dfrac{voters[k]}{2}\Big) \\
& gap \geq 0 \\
\text{minimise} \quad & gap\\
\text{subject to} \quad & \sum_{k=1}^{dist} XA[i,k] = 1 && \forall i, \in 1:Anum \\
& \sum_{k=1}^{dist} XB[i,k] = 1 && \forall i, \in 1:Bnum \\
& (slp[m] \cdot Alist[i,1] + yint[m]) - Alist[i,2] \leq n(1-XA[i,m+1]) - \epsilon \cdot XA[i,m+1] && \forall i \in 1:Anum, \forall m\in 1:(dist-1)\\ 
& (slp[m] \cdot Blist[i,1] + yint[m]) - Blist[i,2] \leq n(1-XB[i,m+1]) - \epsilon \cdot XB[i,m+1] && \forall i \in 1:Bnum, \forall m\in 1:(dist-1)\\ 
& (slp[m] \cdot Alist[i,1] + yint[m]) - Alist[i,2] \geq \epsilon \cdot XA[i,m] - n(1-XA[i,m])&& \forall i \in 1:Anum, \forall m\in 1:(dist-1)\\
& (slp[m] \cdot Blist[i,1] + yint[m]) - Blist[i,2] \geq \epsilon \cdot XB[i,m] - n(1-XB[i,m])&& \forall i \in 1:Bnum, \forall m\in 1:(dist-1)\\
& \Big(\dfrac{vts}{dist}\Big)(1-pd) \leq voters[k] \leq \Big(\dfrac{vts}{dist}\Big)(1+pd) && \forall k=1:dist \\
& vA[k] - vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Bwon[k] && \forall k=1:dist \\
& -vA[k] + vB[k] - 1 \geq -\Big(\dfrac{vts}{dist}\Big)(1+pd)\cdot Awon[k] && \forall k=1:dist \\
& Awon[k] + Bwon[k] = 1 && \forall k=1:dist \\
& gap \geq \dfrac{wA-wB}{vts} \\
& gap \geq \dfrac{wB-wA}{vts}
\end{align}

(Same notation used as above.)

### 2.3. Gerrymandering Objectives ###

If Party A were to gerrymander, they would do so such that they gain the most winning districts as possible. This means that to gerrymander in favour of Party A, we would just have to change the objective to maximise the number of winning districts for A, or
$$ \text{maximise} \quad \sum_{k=1}^{dist} Awon[k] .$$
Similarly, to gerrymander for Party B, the objective would be to $$ \text{maximise} \quad \sum_{k=1}^{dist} Bwon[k].$$

For the block assignment scheme without graphs, the objective also needs to incorporate the objective of making districts connected. So the gerrymandering objective for it would be
$$ \text{maximise} \quad bias\cdot conn + \sum_{k=1}^{dist} Awon[k] $$
or
$$ \text{maximise} \quad bias\cdot conn + \sum_{k=1}^{dist} Bwon[k] $$
depending on party you gerrymander for. Note that the value for $bias$ that will give a good solution may also change from fair voting district objective (it should become lower).

Also note that we did not use the variable $gap$ here because (1) the way it was formulated above it only works when the variable is minimised, and (2) to gerrymander in real life you do not have to worry about the exact efficiency gap, as you're mainly concerned with getting the most seats by winning the most districts.

***

## 3. Solution ##

To obtain a solution, one should run the code in the following order:
1. Run the block below to load all the packages
2. Run the code block with the [global variables](#3.1-Global-Variables) (change values as you wish)
3. Run one of the code block from the [Data Generation section](#3.2-Data-Generation) (to get a sample city and voter distribution)
4. Pick one of the [district drawing scheme](#3.3.-District-Drawing) and run the constraints block for that scheme
5. If you want to gerrymander for a certain party, also run the corresponding objective for the party you want to gerrymander for (default objective is to create fair voting districts).
6. Run the solve block, and wait for the results (time taken could be in the scale of several minutes depending on size of problem)

A couple of sidenotes: 
- For models with large number of variables (i.e. a larger value for $blk$, $Anum$, $Bnum$ or $dist$) the solving time could be long. From what I found, $blk$ or around 8 or more will already take the block scheme a couple of minutes to solve. Similarly, $Anum + Bnum$ of around 100 or more will take the line scheme a couple of minutes to solve.
- Some problems will be infeasible (especially if solved with the block model with small $blk$). If this happens, try having more voters in the city, increase $blk$, or have a higher percentage difference (i.e. increase $pd$).

In [ ]:
# Run this block to load all the relevant Julia packages
using JuMP, Gurobi, PyPlot

### 3.1 Global Variables ###

These are all of the parameters involved in the question. You can change it as you wish.

In [ ]:
# The dimension of the city
n = 30

# The number of voters of Party A
# This should be an integer
Anum = 60

# The number of voters of Party B
# This should be an integer
Bnum = 40

# Cluster centre and standard distribution for both parties for clustered voters distribution (see Model section)
sdvA = 20
xA = 0
yA = 0
sdvB = 20
xB = 30
yB = 30

# Number of districts wanted
# This should be an integer
dist = 3

# The percentage error for size of district
# This should be a positive number (usually between 0 and 1)
pd = 0.2

# For the block scheme - how many blocks wanted (per side)
# Too little blocks may make problem infeasible because of constraint on district size
# This should be an integer
blk = 6

# For the block scheme w/o graph - the bias factor for connectedness of block
# Through experiments, bias factor of around 0.1 and above will guarantee connected district
# Lower bias factor will give better objective value, but may give disconnected districts
# If you get disconnected sections in results try increasing this factor
bias = 1

# A small number
# This is used by logic constraints
ep = 1e-5


####################
# Display purposes #
####################

# Colours used for painting and labelling districts
# Make sure that the length of the list is greater than the number of districts
colours = ["yellow", "cyan", "pink", "white", "lime"]
txtcol = "black"

# Markers to distinguish voters in different districts
# Make sure that the length of this list is greater than the number of districts
mark = ["o", "s", "^", "P", "H"]

# Colours used for the two types of voters
colA = "blue"
colB = "red"


#################################################
#       Automatically generated variables       #
# Code below here should not be touched by user #
#################################################

# Variable which counts the total number of voters
vts = Anum + Bnum

# For the block scheme - variable for the size of the blocks
bsize = n/blk

;

### 3.2 Data Generation ###

The codes here is for generating the dataset that will be used. 

#### 3.2.1 Clustered Voters Distribution ####

In [ ]:
# List to store voters location
Alist = zeros(Anum, 2)
Blist = zeros(Bnum, 2)

figure(figsize=(6,6))
axis("equal")
axis([0,n,0,n])

# Generating each voters location
A_count = 0
while A_count != Anum
    x = sdvA*randn() + xA
    y = sdvA*randn() + yA
    if 0 <= x <= n && 0 <= y <= n
        A_count += 1
        Alist[A_count, 1] = x
        Alist[A_count, 2] = y
        scatter(x, y, color=colA)
    end
end
B_count = 0
while B_count != Bnum
    x = sdvB*randn() + xB
    y = sdvB*randn() + yB
    if 0 <= x <= n && 0 <= y <= n
        B_count += 1
        Blist[B_count, 1] = x
        Blist[B_count, 2] = y
        scatter(x, y, color=colB)
    end
end

xlim(0,n)
ylim(0,n)
show()

#### 3.2.2 Random Voters Distribution ####

In [ ]:
# List to store voters location
Alist = zeros(Anum, 2)
Blist = zeros(Bnum, 2)

figure(figsize=(6,6))
axis("equal")
axis([0,n,0,n])

# Generating each voters location
A_count = 0
while A_count != Anum
    x = rand()*n
    y = rand()*n
    A_count += 1
    Alist[A_count, 1] = x
    Alist[A_count, 2] = y
    scatter(x, y, color=colA)
end
B_count = 0
while B_count != Bnum
    B_count += 1
    x = rand()*n
    y = rand()*n
    Blist[B_count, 1] = x
    Blist[B_count, 2] = y
    scatter(x, y, color=colB)
end

show()

### 3.3. District Drawing ###

To quickly jump to a particular scheme:
- [Block Assignment Scheme without Graphs](#3.3.1.-Block-Assignment-Scheme-without-Graphs)
- [Block Assignment Scheme with Graphs](#3.3.2.-Block-Assignment-Scheme-with-Graphs)
- [Line Drawing Scheme](#3.3.3.-Line-Drawing-Scheme)

#### 3.3.1. Block Assignment Scheme without Graphs ####

##### Creating the model and constraints #####

In [ ]:
# Part for creating blocks and count number of voters in a block
Ablock = zeros(blk,blk)
for i in 1:Anum
    Ablock[trunc(Int8, Alist[i,1]/bsize)+1, trunc(Int8, Alist[i,2]/bsize)+1] += 1
end
Bblock = zeros(blk,blk)
for i in 1:Bnum
    Bblock[trunc(Int8, Blist[i,1]/bsize)+1, trunc(Int8, Blist[i,2]/bsize)+1] += 1
end


# Model for block assignment scheme
ba = Model(solver=GurobiSolver(OutputFlag=0))

# X[i,j,k] = 1 if block at coordinate (i,j) is in district k, 0 otherwise
@variable(ba, X[1:blk,1:blk,1:dist], Bin)

# Constraint on connectivity of districts
@variable(ba, xedge[1:blk-1,1:blk,1:dist], Bin)
@constraint(ba, [i=1:blk-1,j=1:blk,k=1:dist], 2*xedge[i,j,k] <= X[i,j,k] + X[i+1,j,k])
@variable(ba, yedge[1:blk,1:blk-1,1:dist], Bin)
@constraint(ba, [i=1:blk,j=1:blk-1,k=1:dist], 2*yedge[i,j,k] <= X[i,j,k] + X[i,j+1,k])
@expression(ba, conn, sum(xedge) + sum(yedge))

# A block must be in one of the districts
@constraint(ba, [i=1:blk,j=1:blk], sum(X[i,j,:]) == 1)

# Counting voters for each district
@expression(ba, vA[i=1:dist], sum(X[:,:,i] .* Ablock[:,:]))
@expression(ba, vB[i=1:dist], sum(X[:,:,i] .* Bblock[:,:]))
@expression(ba, voters[i=1:dist], vA[i] + vB[i])

# Constraint on total size of the district
@constraint(ba, sizemax[d=1:dist], voters[d] <= vts * (1+pd)/dist)
@constraint(ba, sizemin[d=1:dist], voters[d] >= vts * (1-pd)/dist)

# Variable for winner of each districts
@variable(ba, Awon[i=1:dist], Bin)
@variable(ba, Bwon[i=1:dist], Bin)

# To find winner in each local elections (minus 1 so there is no tie)
@constraint(ba, [i=1:dist], vA[i] - vB[i] - 1 >= -Bwon[i] * vts * (1+pd)/dist)
@constraint(ba, [i=1:dist], -vA[i] + vB[i] - 1 >= -Awon[i] * vts * (1+pd)/dist)
@constraint(ba, Awon .+ Bwon .== 1)

# To find amount of wasted votes
@expression(ba, wA, sum([Awon[i]*(vA[i] - voters[i]/2) + Bwon[i]*vA[i] for i in 1:dist]))
@expression(ba, wB, sum([Bwon[i]*(vB[i] - voters[i]/2) + Awon[i]*vB[i] for i in 1:dist]))

# Efficiency gap
@variable(ba, gap >= 0)
@constraint(ba, gap >= (wA - wB)/vts)
@constraint(ba, gap >= (wB - wA)/vts)

# Fair voting district
@objective(ba, Min, gap - bias*conn)

;

##### Gerrymandering objectives #####

In [ ]:
# Gerrymandering for Party A
@objective(ba, Max, sum(Awon) + bias*conn);

In [ ]:
# Gerrymandering for Party B
@objective(ba, Max, sum(Bwon) + bias*conn);

##### Solving the problem and drawing diagram #####

In [ ]:
@time solve(ba)

# Output stats
println()
println("Party A voter distribution = ", getvalue(vA))
println("Party B voter distribution = ", getvalue(vB))
println()
println("Party A won = ", getvalue(Awon))
println("Party B won = ", getvalue(Bwon))
println()
println("Efficiency Gap = ", abs(getvalue(wA)-getvalue(wB))/vts)
println("conn = ", getvalue(conn))

xans = getvalue(X)

figure(figsize=(7,7))
axis("equal")
axis([0,n,0,n])

# Drawing the districts
txtal = zeros(dist)
for i in 1:blk
    for j in 1:blk
        for k in 1:dist
            if xans[i,j,k] > 0.5
                fill_between([(i-1)*bsize,i*bsize],[(j-1)*bsize,(j-1)*bsize],[j*bsize,j*bsize],color=colours[k])
                if txtal[k] == 0
                    text((i-0.5)*bsize,(j-0.5)*bsize,string("Dist ", k),horizontalalignment="center",fontsize=14)
                    txtal[k] = 1
                end
            end
        end
    end
end

# Plotting the voters
for i in 1:Anum
    xb = trunc(Int8, Alist[i,1]/bsize)+1
    yb = trunc(Int8, Alist[i,2]/bsize)+1
    for k in 1:dist
        if xans[xb, yb, k] > 0.5
            scatter(Alist[i,1], Alist[i,2], color=colA, marker=mark[k])
        end
    end
end
for i in 1:Bnum
    xb = trunc(Int8, Blist[i,1]/bsize)+1
    yb = trunc(Int8, Blist[i,2]/bsize)+1
    for k in 1:dist
        if xans[xb, yb, k] > 0.5
            scatter(Blist[i,1], Blist[i,2], color=colB, marker=mark[k])
        end
    end
end

show()

[Go back and pick another scheme](#3.3.-District-Drawing)

#### 3.3.2. Block Assignment Scheme with Graphs ####

##### Creating the model and constraints #####

In [ ]:
# Part for creating blocks and count number of voters in a block
Ablock = zeros(blk,blk)
for i in 1:Anum
    Ablock[trunc(Int8, Alist[i,1]/bsize)+1, trunc(Int8, Alist[i,2]/bsize)+1] += 1
end
Bblock = zeros(blk,blk)
for i in 1:Bnum
    Bblock[trunc(Int8, Blist[i,1]/bsize)+1, trunc(Int8, Blist[i,2]/bsize)+1] += 1
end


# Model for block assignment scheme
ba2 = Model(solver=GurobiSolver(OutputFlag=0))

# X[i,j,k] = 1 if block at coordinate (i,j) is in district k, 0 otherwise
@variable(ba2, X[1:blk,1:blk,1:dist], Bin)

# A block must be in one of the districts
@constraint(ba2, [i=1:blk,j=1:blk], sum(X[i,j,:]) == 1)

# Edges of district graph
@variable(ba2, xedge[1:blk-1,1:blk,1:dist], Bin)
@constraint(ba2, [i=1:blk-1,j=1:blk,k=1:dist], 2*xedge[i,j,k] <= X[i,j,k] + X[i+1,j,k])
@variable(ba2, yedge[1:blk,1:blk-1,1:dist], Bin)
@constraint(ba2, [i=1:blk,j=1:blk-1,k=1:dist], 2*yedge[i,j,k] <= X[i,j,k] + X[i,j+1,k])

# The flow fraom each edge of graph (for cycle detection)
@variable(ba2, flow[1:blk,1:blk,1:dist,1:4] >= 0)
@constraint(ba2, [i=1:blk-1,j=1:blk,k=1:dist], 2*xedge[i,j,k] == flow[i,j,k,1] + flow[i+1,j,k,2])
@constraint(ba2, [i=1:blk,j=1:blk-1,k=1:dist], 2*yedge[i,j,k] == flow[i,j,k,3] + flow[i,j+1,k,4])
@constraint(ba2, [i=1:blk,j=1:blk,k=1:dist], sum(flow[i,j,k,:]) <= 2-ep)

# Constraint on number of edges in the graph
@constraint(ba2, [i=1:dist], sum(xedge[:,:,i]) + sum(yedge[:,:,i]) == sum(X[:,:,i]) - 1)

# Number of voters in each district
@expression(ba2, vA[i=1:dist], sum(X[:,:,i] .* Ablock[:,:]))
@expression(ba2, vB[i=1:dist], sum(X[:,:,i] .* Bblock[:,:]))
@expression(ba2, voters[i=1:dist], vA[i] + vB[i])

# Constraint on total size of the district
@constraint(ba2, sizemax[d=1:dist], voters[d] <= vts * (1+pd)/dist)
@constraint(ba2, sizemin[d=1:dist], voters[d] >= vts * (1-pd)/dist)

# Variable for winner of each district
@variable(ba2, Awon[i=1:dist], Bin)
@variable(ba2, Bwon[i=1:dist], Bin)

# To find winner in each local elections (minus 1 so there is no tie)
@constraint(ba2, [i=1:dist], vA[i] - vB[i] - 1 >= -Bwon[i] * vts * (1+pd)/dist)
@constraint(ba2, [i=1:dist], -vA[i] + vB[i] - 1 >= -Awon[i] * vts * (1+pd)/dist)
@constraint(ba2, Awon .+ Bwon .== 1)

# To find amount of wasted votes
@expression(ba2, wA, sum([Awon[i]*(vA[i] - voters[i]/2) + Bwon[i]*vA[i] for i in 1:dist]))
@expression(ba2, wB, sum([Bwon[i]*(vB[i] - voters[i]/2) + Awon[i]*vB[i] for i in 1:dist]))

# Efficiency gap
@variable(ba2, gap >= 0)
@constraint(ba2, gap >= (wA - wB)/vts)
@constraint(ba2, gap >= (wB - wA)/vts)

# Fair voting district objective
@objective(ba2, Min, gap)

;

##### Gerrymandering objectives #####

In [ ]:
# Gerrymandering for Party A
@objective(ba2, Max, sum(Awon));

In [ ]:
# Gerrymandering for Party B
@objective(ba2, Max, sum(Bwon));

##### Solving the problem and drawing diagram #####

In [ ]:
@time solve(ba2)

# Output stats
println()
println("Party A voter distribution = ", getvalue(vA))
println("Party B voter distribution = ", getvalue(vB))
println()
println("Party A won = ", getvalue(Awon))
println("Party B won = ", getvalue(Bwon))
println()
println("Efficiency Gap = ", abs(getvalue(wA)-getvalue(wB))/vts)

xans = getvalue(X)

# Drawing the districts
figure(figsize=(6,6))
axis("equal")
axis([0,n,0,n])

txtal = zeros(dist)
for i in 1:blk
    for j in 1:blk
        for k in 1:dist
            if xans[i,j,k] > 0.5
                fill_between([(i-1)*bsize,i*bsize],[(j-1)*bsize,(j-1)*bsize],[j*bsize,j*bsize],color=colours[k])
                if txtal[k] == 0
                    text((i-0.5)*bsize,(j-0.5)*bsize,string("Dist ", k),horizontalalignment="center",fontsize=14)
                    txtal[k] = 1
                end
            end
        end
    end
end

# Plotting the voters
for i in 1:Anum
    xb = trunc(Int8, Alist[i,1]/bsize)+1
    yb = trunc(Int8, Alist[i,2]/bsize)+1
    for k in 1:dist
        if xans[xb, yb, k] > 0.5
            scatter(Alist[i,1], Alist[i,2], color=colA, marker=mark[k])
        end
    end
end
for i in 1:Bnum
    xb = trunc(Int8, Blist[i,1]/bsize)+1
    yb = trunc(Int8, Blist[i,2]/bsize)+1
    for k in 1:dist
        if xans[xb, yb, k] > 0.5
            scatter(Blist[i,1], Blist[i,2], color=colB, marker=mark[k])
        end
    end
end

show()

[Go back and pick another scheme](#3.3.-District-Drawing)

#### 3.3.3. Line Drawing Scheme ####

##### Creating Model and Constraints #####

In [ ]:
la = Model(solver=GurobiSolver(OutputFlag=0))

# X[i,j,k] = 1 if block at coordinate (i,j) is in district k, 0 otherwise
@variable(la, XA[1:Anum,1:dist], Bin)
@variable(la, XB[1:Bnum,1:dist], Bin)

# Position of boundary lines
@variable(la, yin[1:dist-1])
@variable(la, slp[1:dist-1])
@constraint(la, [i=1:dist-2], yin[i] <= yin[i+1])
@constraint(la, [i=1:dist-2], yin[i] + slp[i]*n <= yin[i+1] + slp[i+1]*n)

# Determine which district each voters fall in
for v in 1:Anum
    @constraint(la, [i=1:dist-1], (slp[i]*Alist[v,1] + yin[i]) - Alist[v,2] <= n*(1-XA[v,i+1]) - ep*XA[v,i+1])
    @constraint(la, [i=1:dist-1], (slp[i]*Alist[v,1] + yin[i]) - Alist[v,2] >= ep*XA[v,i] - n*(1-XA[v,i]))
end
for v in 1:Bnum
    @constraint(la, [i=1:dist-1], (slp[i]*Blist[v,1] + yin[i]) - Blist[v,2] <= n*(1-XB[v,i+1]) - ep*XB[v,i+1])
    @constraint(la, [i=1:dist-1], (slp[i]*Blist[v,1] + yin[i]) - Blist[v,2] >= ep*XB[v,i] - n*(1-XB[v,i]))
end

# Total voters
@expression(la, vA[i=1:dist], sum(XA[:,i]))
@expression(la, vB[i=1:dist], sum(XB[:,i]))
@expression(la, voters[i=1:dist], vA[i] + vB[i])

# A voter must be in one of the districts
@constraint(la, [i=1:Anum], sum(XA[i,:]) == 1)
@constraint(la, [i=1:Bnum], sum(XB[i,:]) == 1)

# Constraint on total size of the district
@constraint(la, [d=1:dist], voters[d] <= vts * (1+pd)/dist)
@constraint(la, [d=1:dist], voters[d] >= vts * (1-pd)/dist)

# Variable for whether a party won at a certain district
@variable(la, Awon[i=1:dist], Bin)
@variable(la, Bwon[i=1:dist], Bin)

# To find winner in each local elections (minus 1 so there is no tie)
@constraint(la, [i=1:dist], vA[i] - vB[i] - 1 >= -Bwon[i] * vts * (1+pd)/dist)
@constraint(la, [i=1:dist], -vA[i] + vB[i] - 1 >= -Awon[i] * vts * (1+pd)/dist)
@constraint(la, Awon .+ Bwon .== 1)

# To find amount of wasted votes
@expression(la, wA, sum([Awon[i]*(vA[i] - voters[i]/2) + Bwon[i]*vA[i] for i in 1:dist]))
@expression(la, wB, sum([Bwon[i]*(vB[i] - voters[i]/2) + Awon[i]*vB[i] for i in 1:dist]))

# Efficiency gap
@variable(la, gap >= 0)
@constraint(la, gap >= (wA - wB)/vts)
@constraint(la, gap >= (wB - wA)/vts)

# Fair voting district
@objective(la, Min, gap)

;

##### Gerrymandering objectives #####

In [ ]:
# Gerrymandering for Party A
@objective(la, Max, sum(Awon));

In [ ]:
# Gerrymandering for Party B
@objective(la, Max, sum(Bwon));

##### Solving the problem and drawing diagrams #####

In [ ]:
@time solve(la)

# Output stats
println()
println("Party A voter distribution = ", getvalue(vA))
println("Party B voter distribution = ", getvalue(vB))
println()
println("Party A won = ", getvalue(Awon))
println("Party B won = ", getvalue(Bwon))
println()
println("Efficiency Gap = ", abs(getvalue(wA)-getvalue(wB))/vts)

slpval = getvalue(slp)
yinval = getvalue(yin)
xA_lst = getvalue(XA)
xB_lst = getvalue(XB)

figure(figsize=(6,6))
axis("equal")
axis([0,n,0,n])

# Plotting the districts
fill_between([0,n],[0,0],[n,n],color=colours[dist])
for i in dist-1:-1:1
    a = slpval[i]
    b = yinval[i]
    fill_between([0,n], [0,0], [b, a*n+b], color=colours[i])
end
ylocn = zeros(dist+1)
ylocn[end] = n
for i in 1:dist-1
    ylocn[i+1] = yinval[i] + (n*0.5)*slpval[i]
end
for i in 1:dist
    text(n*0.5,(ylocn[i]+ylocn[i+1])*0.5,string("Dist ", i),horizontalalignment="center",
        verticalalignment="center",fontsize=14)
end

# Plotting the voters
for i in 1:Anum
    for d in 1:dist
        if xA_lst[i,d] > 0.5
            scatter(Alist[i,1], Alist[i,2], color=colA, marker=mark[d])
        end
    end
end
for i in 1:Bnum
    for d in 1:dist
        if xB_lst[i,d] > 0.5
            scatter(Blist[i,1], Blist[i,2], color=colB, marker=mark[d])
        end
    end
end

show()

[Go back and pick another scheme](#3.3.-District-Drawing)

***

## 4. Results and Discussion ##

### 4.1. Geometry of Solutions ###

In this section we discuss the general shape of solutions obtained from the different models without considering performances in solving the problem. The three solving schemes were ran against some sample data both with random voters and clustered voters. Some of the results are as below (blue dots represents voters of Party A, red dot represents voters of Party B).

<img src="img/restable1.png">

From the results, we can roughly see that the block scheme with graphs tend to produce a "stranger" and less convex looking shape. This is the result of the nature of the model which doesn't consider the overall shape of the districts, just whether the shapes made will fit the constraint. This can be contrasted with the block scheme without graphs which has got an objective that promotes neighbouring blocks to be of the same district (hence produces more rounded groups of blocks in the same district), or with the line drawing scheme which will produce convex solutions anyway due to the geometry of straight lines.

### 4.2. Quantitative Comparisons of Different Methods ###

#### 4.2.1. Comparison of the Block Models ####

Since the time taken to solve the problem with the two block schemes (with and without graph) is expected to depend on the value of $blk$ (i.e. number of constraints and number of variables will depend on $blk$), we can try to compare their performances directly. 

To test the speed of the two graph schemes, we can run the model for different values of the available parameters. We let the parameters be the following.

- Use the clustered voter distribution,
- $n$ be values $[15, 20, 25, 30, 35]$,
- $Anum = 2n^2/5$ and $Bnum = Anum/2$ (where we assume that population density of the city is about the same, so population scales with area, or with $n^2$, and we also let there be a majority party and a minority party),
- $dist=3$,
- $pd = 0.2$,
- $sdv_A = sdv_B = n/2$,
- $x_{A}=y_{A}=n/5$ and $x_{B}=y_{B}=4n/5$,
- $blk = n/5$ which means that the blocks in all samples will have a size of 5, which is a block small enough that we generally will get no infeasible problems.

We run each model with a certain parameter value 10 times and take the average. The graph of the results is as shown below.

<img src="img/res1.png" width="600">

As we can see, the block scheme which doesn't use graphs tend to be quicker than the scheme using graphs. However, what was surprising was that the speed for both models seem to be increasing at around the same exponential rate (as seen by the rough slope of the graph). This shows that while using graphs does result in longer computation times, it is not that much longer (less than an order of magnitude longer).

The graph also seems like it may intersect at a point, which may suggest that there is a point where the model with graphs will overtake the model without graphs. However this shouldn't be drawn as a definite conclusion until further analysis is done.

It is worth mentioning that all 10 iterations of the models for certain parameters were run all in one sitting, which may affect how fast the code runs (compared to if the notebook was started fresh and the code is then run, for example).

We should also note that it is harder to compare the graph schemes against the line scheme though, since the time to solve the problem using the line scheme mainly depend on the number of total voters (which directly influences the number of constraints). We expect that the time to solve the problem using the line scheme will grow much quicker than the time used to solve with the block scheme since in real life, the total number of voters would grow quicker than the number of blocks we choose to divide the voters into.

#### 4.2.2. Efficiency Gap ####

We can also test to see which models is able to complete the objective the best. To test this, we can use the same parameters as above and run it against all of the three models (we can now compare all three methods directly since we are not interested in the time it takes to come up with such values, only their efficiency gap). Here are the results we achieve.

| $n$ | Block w/o Graph | Block w/ Graph | Line |
|-----|-----------------|----------------|------|
| 15  | 0.151 | 0.120 | 0.100 |
| 20  | 0.159 | 0.100 | 0.100 |
| 25  | 0.169 | 0.100 | 0.100 |
| 30  | 0.129 | 0.100 | 0.100 |
| 35  | 0.165 | 0.100 | 0.100 |


We can see that the efficiency gap achieved by block model with graph and by the line model is mostly the same for the different cases. This suggests that the two schemes here tend to produce the fairest districts, and that there isn't a specific scheme that gives a better answer than the other but rather both will give equally good answers as long as you give the districts drawn enough degrees of freedom to work with.

However there are a couple of points that should be mentioned about this conclusion. Firstly, in the block model without graphs, a better value for $bias$ could have been chosen to get a better efficiency gap (see next section). We have not taken this fact into consideration for this part.

Secondly, in reality, the actual amount of efficiency gap will not matter too much. In real elections, what matters most is that the demographics of the population is represented equally, which would be fulfilled if the proprtion of winning district matches as close to the proportion of population as possible. All three schemes produces a solution that is "good enough", and so to get the lowest possible efficiency gap would not be necessary in real life applications.

#### 4.2.3. Variation of $bias$ on solution of graph method without graphs ####

We look at the effect of varying $bias$ on the solutions we get in the block scheme which doesn't use graphs. To analyse the varying effect of $bias$, we generated a voter distribution and tried finding a district to fit it while varying $bias$. The results we achieve is as follows.

<img src="img/restable3.png">

<img src="img/biasplot.png">

The graph above plots the values of $conn$ and $gap$ we get when we adjust $bias$ to different amounts (note we omit the point for $bias=0$ which has $conn$ of around 20, meaning all the points in the graph above represent good solutions with connected districts). The $conn$ factor we get do not vary too much between the different values of $bias$ that we choose. The possible objective values we get are discrete (meaning a certain range of $bias$ will just give the same solution) which is expected with integer variables.

We can note a couple of things from this.
- When the value of $bias$ is zero, the district that is drawn will have minimal efficiency gap (which in this case is zero). However, it will usually give a solution which has disconnected districts. This is because the objective doesn't consider the geometry of the districts at all because the objective function does not require it to.
- As the value of $bias$ becomes higher, we get district shapes that are "nicer" and more block-shaped. By the time we get to $bias=2$ the district shape that we get is very simple, and could have been defined using two straight lines. However, the efficiency gap that we achieved also goes up and we get a district that is less fair.
- For our case, there does exist a value of $bias$ which creates districts that are connected and also achieve minimal (not necessarily zero) effiency gap, which in this case is $bias=0.01$. This shows that this scheme can in fact give a solution as good as the other schemes given that we pick the right value of $bias$. We do not have a method to precisely determine a good value of $bias$ for a given problem rather than via experimentation or by estimation, but the fact that such value for $bias$ exists shows that this method can be promising.

### 4.3. Discussion on Different Schemes ###

To summarise the performance of the different schemes, we can summarise the points below.

_Block Scheme without Graphs_
- Advantages
    - Quicker than using graphs
    - Easy to form blocks in real life
    - Solution tends to be "more convex" than using graphs
- Disadvantages
    - Higher efficiency gap than the other schemes if chosen a bad value of $bias$
    - Requires finding the right value of $bias$ to get a good solution (balance between connected district and low efficiency gap)

_Block Scheme with Graphs_
- Advantages
    - Can get the minimal efficiency gap
    - Easy to form blocks in real life
- Disadvantages
    - Longer runtime than scheme without using graphs
    - Can produce less convex shapes (in real life, this could lead to districts that may be trickier to enforce)

_Line Scheme_
- Advantages
    - Can get minimal efficiency gap
    - Geometry of district relatively simple
- Disadvantages
    - Optimisation model works with exact location of voters (which is a data that may not always be available)
    - Scales with total population which is a quantity that cannot be changed in real life (unlike sizes of blocks which we choose), and could be very large
    - Shapes of district is not as practical because it is shaped like narrow strips especially with more districts (this is rather subjective though)
    
Qualitatively, we find that the block schemes seem to be the more promising methods for district lines. For the block schemes involving graphs, we tend to get optimal solutions with minimal efficiency gap. When we do not use graphs, while we may not get solutions as good, we do save time and also get nicer looking solutions. Most importantly, the two graph schemes both do not scale with the number of population but only with the variable $blk$, which means it can be adjusted to work better as we have models with mroe people (e.g. in real cities).

### 4.4. Gerrymandering ###

A test population model was used and the objectives of the models were to draw districts in favour of one of the parties. The results were as follows.

<img src="img/restable2.png">

The $bias$ factor was lowered as much as possible such that the solution given contains no separated district (note that the lowest $bias$ we get before we get invalid districts is higher than $bias$ we get from Section 4.2.3.). 

We can see that the block scheme with graphs performs the best under these objectives. This was to be expected because this scheme has higher number of degrees of freedom compared to the line model, and was less restricted into drawing "stranger" district shapes, hence was able to draw itself to fit best with objectives we get. However I do not believe there are no particular reason why some model does not maximise gerrymandering in some objectives other than the nature of the chosen population model.

What is also interesting is that we can see clear attempts of packing and cracking when gerrymandering succeeds. Since Party A is a majority, cracking would improve their representation, which is what happens in the top row. Alternatively, Party B is the minority, and packing voters of Party A is a strategy that would help them gain a majority, which is what happens in the bottom row (see district 1 of the two block schemes, where many Party A voters are).

***

## 5. Conclusion ##

In this report, we have explored different geometries that we can use in order to draw voting districts that is fair and representative of the voters. We have used schemes of dividing voters into blocks that would belong to some certain district, and also a scheme where we divide the town using straight lines to allocate voters to different districts. We also discussed the performances of the different models, and more qualitative reasons why some models are better than others.

Out of all the models explored, the block assignment models seems to be a good start if we were to implement such algorithms in real life examples. We can implement block schemes which is relatively fast and doesn't grow too quickly with size of problem (if we didn't use graphs), or get fairer districts (if we used graphs). 

In future works, a combination of techniques from graph and non-graph schemes can be used, so that we can ensure our districts are connected and valid, while also being a more convex-looking shape. 

We can also look for ways to refine our schemes, or come up with a new scheme altogether, which will scale better with larger towns and will be able to run faster. This may include methods such as not having to minimise efficiency gap, but making representation match as close as possible to voter proportion, which will give solutions that is "good enough" for real life. 

We can also extend the models we have beyond bi-partisan systems, but onto systems where there are more than two political parties, which is the case in many democratic countries outside the United States.

***

## 6. Sources ##

- *Gerrymandering* from Wikipedia: https://en.wikipedia.org/wiki/Gerrymandering
- *Solutions to Gerrymandering* by Lauren Payne-Riley: https://www.policymap.com/2017/08/solutions-to-gerrymandering/
- *How the Efficiency Gap Works* by the Brennan Center: https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf
- Cycle detection as a linear program adapted from *Several Graph problems and their Linear Program formulations* by Nathan Cohen: https://hal.inria.fr/inria-00504914/document
- Also, not really a source used for this project, but a video by CGP Grey on gerrymandering: https://www.youtube.com/watch?v=Mky11UJb9AY